In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.tree import DecisionTreeRegressor

In [26]:
df_origin = pd.read_csv('../Datasets/card_cust.csv')
df_origin.head()
df_origin['CREDIT_LIMIT'] = df_origin['CREDIT_LIMIT'].fillna(df_origin['CREDIT_LIMIT'].mean())
df_origin['MINIMUM_PAYMENTS'] = df_origin['MINIMUM_PAYMENTS'].fillna(df_origin['MINIMUM_PAYMENTS'].mean())

# 문제 1

In [29]:
df1 = df_origin.copy()
df_origin.columns

Index(['CUST_ID', 'BALANCE', 'BALANCE_FREQUENCY', 'PURCHASES',
       'ONEOFF_PURCHASES', 'INSTALLMENTS_PURCHASES', 'CASH_ADVANCE',
       'PURCHASES_FREQUENCY', 'ONEOFF_PURCHASES_FREQUENCY',
       'PURCHASES_INSTALLMENTS_FREQUENCY', 'CASH_ADVANCE_FREQUENCY',
       'CASH_ADVANCE_TRX', 'PURCHASES_TRX', 'CREDIT_LIMIT', 'PAYMENTS',
       'MINIMUM_PAYMENTS', 'PRC_FULL_PAYMENT', 'TENURE'],
      dtype='object')

연간 평균 잔고액 BALANCE

신용카드 서비스 이용기간 TENURE

간의 관계를 파악하여 신용카드 한도 CREDIT_LIMIT 조정에 근거 자료로 활용

In [30]:
df1_cat = ['BALANCE', 'TENURE', 'CREDIT_LIMIT']

In [32]:
df1_cut = df1[df1_cat]

In [34]:
round(df1_cut.groupby('TENURE').corr(),2)

BALANCE  CREDIT_LIMIT
TENURE                                    
6.0    BALANCE          1.00          0.87
       CREDIT_LIMIT     0.87          1.00
7.0    BALANCE          1.00          0.95
       CREDIT_LIMIT     0.95          1.00
8.0    BALANCE          1.00          0.82
       CREDIT_LIMIT     0.82          1.00
9.0    BALANCE          1.00          0.09
       CREDIT_LIMIT     0.09          1.00
10.0   BALANCE          1.00          0.29
       CREDIT_LIMIT     0.29          1.00
11.0   BALANCE          1.00          0.38
       CREDIT_LIMIT     0.38          1.00
12.0   BALANCE          1.00          0.46
       CREDIT_LIMIT     0.46          1.00

> A.  TENURE = 7.0  일 때, 0.95

In [163]:
round(df1_cut.groupby('TENURE').corr(),2).unstack()['BALANCE']['CREDIT_LIMIT'].max()

0.95

# Solution

In [154]:
df1_s = df_origin.copy()

In [156]:
df1_s[['BALANCE','CREDIT_LIMIT']].corr()

BALANCE  CREDIT_LIMIT
BALANCE       1.000000      0.467646
CREDIT_LIMIT  0.467646      1.000000

In [158]:
ls_corr=[]
for n in df1['TENURE'].unique():
    _df = df1_s.loc[df1_s['TENURE']==n,['BALANCE','CREDIT_LIMIT']]
    _stat_corr = _df.corr().iloc[0,1]
    ls_corr.append(_stat_corr)

In [160]:
max(ls_corr).round(2)

0.95

# 문제 2

In [35]:
df2 = df_origin.copy()

In [38]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   CUST_ID                           1000 non-null   int64  
 1   BALANCE                           1000 non-null   float64
 2   BALANCE_FREQUENCY                 1000 non-null   float64
 3   PURCHASES                         1000 non-null   float64
 4   ONEOFF_PURCHASES                  1000 non-null   float64
 5   INSTALLMENTS_PURCHASES            1000 non-null   float64
 6   CASH_ADVANCE                      1000 non-null   float64
 7   PURCHASES_FREQUENCY               1000 non-null   float64
 8   ONEOFF_PURCHASES_FREQUENCY        1000 non-null   float64
 9   PURCHASES_INSTALLMENTS_FREQUENCY  1000 non-null   float64
 10  CASH_ADVANCE_FREQUENCY            1000 non-null   float64
 11  CASH_ADVANCE_TRX                  1000 non-null   float64
 12  PURCHAS

In [50]:
df2_drop = df2.drop('CUST_ID',axis=1)

In [51]:
df2_drop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   BALANCE                           1000 non-null   float64
 1   BALANCE_FREQUENCY                 1000 non-null   float64
 2   PURCHASES                         1000 non-null   float64
 3   ONEOFF_PURCHASES                  1000 non-null   float64
 4   INSTALLMENTS_PURCHASES            1000 non-null   float64
 5   CASH_ADVANCE                      1000 non-null   float64
 6   PURCHASES_FREQUENCY               1000 non-null   float64
 7   ONEOFF_PURCHASES_FREQUENCY        1000 non-null   float64
 8   PURCHASES_INSTALLMENTS_FREQUENCY  1000 non-null   float64
 9   CASH_ADVANCE_FREQUENCY            1000 non-null   float64
 10  CASH_ADVANCE_TRX                  1000 non-null   float64
 11  PURCHASES_TRX                     1000 non-null   float64
 12  CREDIT_

In [52]:
zscaler = StandardScaler()

In [58]:
df2_std = zscaler.fit_transform(df2_drop)

In [67]:
seed = 1234
for k in [2,3,4,5]:
    model_kmeans = KMeans(n_clusters=k,random_state=seed)
    model_kmeans.fit(df2_std)
    print(round(silhouette_score(df2_std,model_kmeans.labels_),2))


C:\Users\ksh69\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\ksh69\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


0.31


C:\Users\ksh69\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\ksh69\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


0.2


C:\Users\ksh69\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\ksh69\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


0.21


C:\Users\ksh69\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\ksh69\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


0.19


> k=2 일 때, 제일 높군

In [68]:
model_kmeans = KMeans(n_clusters=2,random_state=seed)
model_kmeans.fit(df2_std)

C:\Users\ksh69\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\ksh69\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


KMeans(n_clusters=2, random_state=1234)

In [80]:
pred = model_kmeans.predict(df2_std)
df2_drop['clust'] = pred
df2_drop

BALANCE  BALANCE_FREQUENCY  PURCHASES  ONEOFF_PURCHASES  \
0      40.900749           0.818182      95.40              0.00   
1    3202.467416           0.909091       0.00              0.00   
2    2495.148862           1.000000     773.17            773.17   
3    1666.670542           0.636364    1499.00           1499.00   
4     817.714335           1.000000      16.00             16.00   
..           ...                ...        ...               ...   
995  1250.394614           0.909091     443.99            443.99   
996     9.503968           1.000000      96.62              0.00   
997  2285.068731           1.000000       0.00              0.00   
998  2928.756699           1.000000     160.92              0.00   
999  1460.178578           1.000000     937.96            100.00   

     INSTALLMENTS_PURCHASES  CASH_ADVANCE  PURCHASES_FREQUENCY  \
0                     95.40      0.000000             0.166667   
1                      0.00   6442.945483             0.000000   
2                      0.00      0.000000             1.000000   
3                      0.00    205.788017             0.083333   
4                      0.00      0.000000             0.083333   
..                      ...           ...                  ...   
995                    0.00      0.000000             0.272727   
996                   96.62      0.000000             1.000000   
997                    0.00   1173.310874             0.000000   
998                  160.92    319.931964             1.000000   
999                  837.96      0.000000             1.000000   

     ONEOFF_PURCHASES_FREQUENCY  PURCHASES_INSTALLMENTS_FREQUENCY  \
0                      0.000000                          0.083333   
1                      0.000000                          0.000000   
2                      1.000000                          0.000000   
3                      0.083333                          0.000000   
4                      0.083333                          0.000000   
..                          ...                               ...   
995                    0.272727                          0.000000   
996                    0.000000                          1.000000   
997                    0.000000                          0.000000   
998                    0.000000                          1.000000   
999                    0.083333                          1.000000   

     CASH_ADVANCE_FREQUENCY  CASH_ADVANCE_TRX  PURCHASES_TRX  CREDIT_LIMIT  \
0                  0.000000               0.0            2.0        1000.0   
1                  0.250000               4.0            0.0        7000.0   
2                  0.000000               0.0           12.0        7500.0   
3                  0.083333               1.0            1.0        7500.0   
4                  0.000000               0.0            1.0        1200.0   
..                      ...               ...            ...           ...   
995                0.000000               0.0            3.0        3500.0   
996                0.000000               0.0           19.0        4500.0   
997                0.166667               5.0            0.0        2500.0   
998                0.333333               5.0           12.0        3000.0   
999                0.000000               0.0           40.0        1500.0   

        PAYMENTS  MINIMUM_PAYMENTS  PRC_FULL_PAYMENT  TENURE  clust  
0     201.802084        139.509787          0.000000    12.0      0  
1    4103.032597       1072.340217          0.222222    12.0      0  
2     622.066742        627.284787          0.000000    12.0      0  
3       0.000000       1297.116322          0.000000    12.0      0  
4     678.334763        244.791237          0.000000    12.0      0  
..           ...               ...               ...     ...    ...  
995   273.823646        259.715939          0.000000    11.0      0  
996  1086.932525         92.217936          0.250000    12.0      0  
997

In [83]:
df2_drop.groupby('clust')['ONEOFF_PURCHASES'].mean().round(2)

clust
0     340.23
1    3946.19
Name: ONEOFF_PURCHASES, dtype: float64

> A.  3946.19

# Solution

In [164]:
#params
seed = 1234

In [177]:
df2_s = df_origin.copy()

In [178]:
s_cust_id = df2_s.pop('CUST_ID')

In [179]:
scaler_sd = StandardScaler()

In [187]:
df2_s_std = scaler_sd.fit_transform(df2_s)

In [188]:
lst_scores = []
for k in range(2,6):
    model_kmeans = KMeans(n_clusters=k,random_state=seed)
    model_kmeans.fit(df2_s_std)
    score_sub = silhouette_score(df2_s_std,model_kmeans.labels_)
    lst_scores.append(score_sub)

C:\Users\ksh69\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\ksh69\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
C:\Users\ksh69\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\ksh69\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less 

In [189]:
lst_scores

[0.3075281530456079,
 0.19636128772937608,
 0.207150984946399,
 0.19274056144483248]

In [193]:
pd.Series(lst_scores,index = range(2,6))

2    0.307528
3    0.196361
4    0.207151
5    0.192741
dtype: float64

In [195]:
best_k = 2

In [196]:
model_kmeans = KMeans(n_clusters=best_k,random_state=seed)
model_kmeans.fit(df2_s_std)

C:\Users\ksh69\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\ksh69\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


KMeans(n_clusters=2, random_state=1234)

In [198]:
df2_s['cluster'] = model_kmeans.labels_

In [202]:
df2_s.groupby('cluster')['ONEOFF_PURCHASES'].mean()

cluster
0     340.230998
1    3946.187525
Name: ONEOFF_PURCHASES, dtype: float64

# 문제 3

In [203]:
df3 = df_origin.copy()

In [109]:
X_train = df3[(df3['CUST_ID'] % 4) != 0]

In [110]:
X_test = df3[(df3['CUST_ID'] % 4) == 0]

In [111]:
X_train

CUST_ID      BALANCE  BALANCE_FREQUENCY  PURCHASES  ONEOFF_PURCHASES  \
0      10001    40.900749           0.818182      95.40              0.00   
1      10002  3202.467416           0.909091       0.00              0.00   
2      10003  2495.148862           1.000000     773.17            773.17   
4      10005   817.714335           1.000000      16.00             16.00   
5      10006  1809.828751           1.000000    1333.28              0.00   
..       ...          ...                ...        ...               ...   
993    11027  2845.160430           0.727273     735.09            652.94   
995    11029  1250.394614           0.909091     443.99            443.99   
996    11030     9.503968           1.000000      96.62              0.00   
997    11031  2285.068731           1.000000       0.00              0.00   
999    11033  1460.178578           1.000000     937.96            100.00   

     INSTALLMENTS_PURCHASES  CASH_ADVANCE  PURCHASES_FREQUENCY  \
0                     95.40      0.000000             0.166667   
1                      0.00   6442.945483             0.000000   
2                      0.00      0.000000             1.000000   
4                      0.00      0.000000             0.083333   
5                   1333.28      0.000000             0.666667   
..                      ...           ...                  ...   
993                   82.15      0.000000             0.666667   
995                    0.00      0.000000             0.272727   
996                   96.62      0.000000             1.000000   
997                    0.00   1173.310874             0.000000   
999                  837.96      0.000000             1.000000   

     ONEOFF_PURCHASES_FREQUENCY  PURCHASES_INSTALLMENTS_FREQUENCY  \
0                      0.000000                          0.083333   
1                      0.000000                          0.000000   
2                      1.000000                          0.000000   
4                      0.083333                          0.000000   
5                      0.000000                          0.583333   
..                          ...                               ...   
993                    0.250000                          0.333333   
995                    0.272727                          0.000000   
996                    0.000000                          1.000000   
997                    0.000000                          0.000000   
999                    0.083333                          1.000000   

     CASH_ADVANCE_FREQUENCY  CASH_ADVANCE_TRX  PURCHASES_TRX  CREDIT_LIMIT  \
0                  0.000000               0.0            2.0        1000.0   
1                  0.250000               4.0            0.0        7000.0   
2                  0.000000               0.0           12.0        7500.0   
4                  0.000000               0.0            1.0        1200.0   
5                  0.000000               0.0            8.0        1800.0   
..                      ...               ...            ...           ...   
993                0.000000               0.0            8.0       10500.0   
995                0.000000               0.0            3.0        3500.0   
996                0.000000               0.0           19.0        4500.0   
997                0.166667               5.0            0.0        2500.0   
999                0.000000               0.0           40.0        1500.0   

        PAYMENTS  MINIMUM_PAYMENTS  PRC_FULL_PAYMENT  TENURE  
0     201.802084        139.509787          0.000000    12.0  
1    4103.032597       1072.340217          0.222222    12.0  
2     622.066742        627.284787          0.000000    12.0  
4     678.334763        244.791237          0.000000    12.0  
5    1400.057770       2407.246035          0.000000    12.0  
..           ...               ...               ...     ...  
993  6006.492582        888.901116          0.000000    12.0  
995   273.823646 

In [119]:
X_train_feature = X_train.drop({'CUST_ID','ONEOFF_PURCHASES'},axis=1)

In [120]:
X_train_target = X_train['ONEOFF_PURCHASES']

In [121]:
X_test_feature =  X_test.drop({'CUST_ID','ONEOFF_PURCHASES'},axis=1)

In [122]:
X_test_targe = X_test['ONEOFF_PURCHASES']

In [216]:
model = DecisionTreeRegressor(random_state=seed)

In [217]:
model.fit(X_train_feature,X_train_target)

DecisionTreeRegressor(random_state=1234)

In [218]:
y_pred = model.predict(X_test_feature)

In [219]:
((((X_test_targe-y_pred)**2).sum())/248)**0.5

1039.193967231063

> # Solution

In [204]:
df3 = df_origin.copy()

In [207]:
seed = 1234
df_test = df3.loc[df3['CUST_ID']%4 == 0]
df_train = df3.loc[df3['CUST_ID']%4 != 0]

In [211]:
len(df_test),len(df_train)

(248, 752)

In [212]:
y_train = df_train.pop('ONEOFF_PURCHASES')
y_test = df_test.pop('ONEOFF_PURCHASES')

In [214]:
X_train = df_train.drop('CUST_ID',axis =1)
X_test = df_test.drop(columns='CUST_ID')

In [215]:
model_dtr = DecisionTreeRegressor(random_state=seed)

In [220]:
model_dtr.fit(X=X_train, y=y_train)

DecisionTreeRegressor(random_state=1234)

In [221]:
y_pred = model_dtr.predict(X_test)

In [223]:
round((((y_test - y_pred)**2).mean())**0.5,1)

1039.2

In [224]:
from sklearn.metrics import mean_squared_error

In [228]:
round(mean_squared_error(y_test,y_pred)**0.5,1)

1039.2